# Introduction
Here I will build the acetone strain, by inserting the acetone production pathways, as well as blocking the LDH reaction, as is done in the strain. 

Then I'll add transport and exchanges for the metabolites that the model cannot make anaerobically, and save the model for use in the further dynamic modeling project. 

In [1]:
import cameo
import pandas as pd
import cobra.io
#import escher
#from escher import Builder
from cobra import Metabolite, Reaction
import numpy as np
from matplotlib import pyplot as plt
from os.path import dirname, join, pardir

In [2]:
data_dir = "/Users/phivri/Documents/GitHubCleanup/MoGeoCoCult/Models"
pth = cobra.io.read_sbml_model(join(data_dir,'p-thermo.xml'))

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


### Acetone pathway

In [3]:
#add the acetone metabolite, both intracellular & extracellular
pth.add_metabolites(Metabolite(id='act_c',formula = 'C3H6O', name = 'Acetone', compartment = 'c' ))

In [4]:
pth.add_metabolites(Metabolite(id='act_e', formula = 'C3H6O', name = 'Acetone', compartment = 'e'))

In [5]:
#then add the reactions one by one
ctfAB_ac = Reaction("ctfAB_ac")
ctfAB_ac.name = "Acetate-acetoacetate CoA-transferase"

ctfAB_ac.add_metabolites({
    pth.metabolites.ac_c: -1.0,      #Acetate
    pth.metabolites.aacoa_c: -1.0,   #Acetoacetyl-CoA
    pth.metabolites.acac_c: 1.0,     #Acetoacetate
    pth.metabolites.accoa_c: 1.0,    #Acetyl-CoA
    pth.metabolites.h_c: 1.0
})
print(ctfAB_ac.check_mass_balance())    #ensure correct mass balance
print(ctfAB_ac.bounds)                  #Ensure correct bounds
print(ctfAB_ac.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
aacoa_c + ac_c --> acac_c + accoa_c + h_c


In [6]:
ctfAB_bu = Reaction("ctfAB_bu")
ctfAB_bu.name = "Butyrate-acetoacetate CoA-transferase"

ctfAB_bu.add_metabolites({
    pth.metabolites.but_c: -1.0,     #Butyrate
    pth.metabolites.aacoa_c: -1.0,   #Acetoacetyl-CoA
    pth.metabolites.acac_c: 1.0,     #Acetoacetate    
    pth.metabolites.btcoa_c: 1.0,    #Butyryl-CoA
    pth.metabolites.h_c: 1.0
})
print(ctfAB_bu.check_mass_balance())    #ensure correct mass balance
print(ctfAB_bu.reaction)                #Ensure correct reaction

{}
aacoa_c + but_c --> acac_c + btcoa_c + h_c


In [7]:
thl = Reaction("THL")
thl.name = "acetyl-CoA C-acetyltransferase"

thl.add_metabolites({
    pth.metabolites.accoa_c: -2.0,     #Acetyl-coa
    pth.metabolites.h_c: -1.0,
    pth.metabolites.aacoa_c: 1.0,     #Acetoacetyl-CoA
    pth.metabolites.coa_c: 1.0
})
print(thl.check_mass_balance())    #ensure correct mass balance
print(thl.reaction)                #Ensure correct reaction

{}
2.0 accoa_c + h_c --> aacoa_c + coa_c


In [8]:
adc = Reaction("ADC")
adc.name = "acetoacetate decarboxylase"

adc.add_metabolites({
    pth.metabolites.acac_c: -1.0,     #Acetoacetate
    pth.metabolites.h_c: -1.0, 
    pth.metabolites.act_c: 1.0,
    pth.metabolites.co2_c: 1.0
})
print(adc.check_mass_balance())    #ensure correct mass balance
print(adc.reaction)                #Ensure correct reaction

{}
acac_c + h_c --> act_c + co2_c


In [9]:
actT = Reaction("actT")
actT.name = "Acetone transporter"
actT.add_metabolites({ 
    pth.metabolites.act_c: -1.0,
    pth.metabolites.act_e: 1.0
})
actT

Reaction identifier,actT
Name,Acetone transporter
Memory address,0x07ff511937be0
Stoichiometry,act_c --> act_e Acetone --> Acetone
GPR,
Lower bound,0.0
Upper bound,1000.0


In [10]:
act_ex = Reaction("EX_act_e")
act_ex.name = "Acetone exchange"
act_ex.add_metabolites({ 
    pth.metabolites.act_e: -1.0,
})
act_ex

Reaction identifier,EX_act_e
Name,Acetone exchange
Memory address,0x07ff511937a90
Stoichiometry,act_e --> Acetone -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
#add all the reactions to the model
pth.add_reactions([ctfAB_ac, ctfAB_bu, thl, adc, actT, act_ex])

In [12]:
#block LDH reaction
pth.reactions.LDH_L.bounds = (0,0)

### Anaerobic production of compounds of interest
We saw in the GSMM paper of Mol et al., (2021, https://doi.org/10.1016/j.ymben.2021.03.002) that there are some metabolites the model cannot synthesize anaerobically, though experimental data proves they can. As we do not have enough information to fix this through adding correct pathways in the model, instead we will add these components to the model medium. This also puts the requirement of needing exhcanges and transports for these compounds that I will code in.

As we do not have information about the way these metabolites are transported across the membrane, i will add them as passive transport. 

In [13]:
#optimize for growth with a little bit of oxygen
#to check objective later is the same
pth.reactions.EX_o2_e.bounds = (-0.1,1000)
pth.optimize().objective_value

0.12613059010111424

In [14]:
#list of metabolites that need to be taken up
mets=['btn','thmpp','sprm','spmd','adocbl','10fthf','fe3','pheme']

In [15]:
for met in mets:
    #create extracellular metabolite
    emet = Metabolite(id=f"{met}_e", 
                      name = f"extracellular {met}", 
                      formula = pth.metabolites.get_by_id( f"{met}_c").formula, #copy formula & charge from intracell met
                      charge = pth.metabolites.get_by_id( f"{met}_c").charge, 
                      compartment ='e')
    pth.add_metabolites(emet)
    #create transport reaction
    rct = Reaction(id=f"{met.upper()}t", name = f"{met} transport")
    #add intra and extra cellular metabolites
    rct.add_metabolites({
        pth.metabolites.get_by_id( f"{met}_c"):-1,
        pth.metabolites.get_by_id( f"{met}_e"):1
    })
    #define transport bounds as irreversible
    rct.bounds = (-1000,1000)
    #Add transport reaction to the model
    pth.add_reaction(rct)
    #create exchange for the extracellular metabolite
    pth.add_boundary(pth.metabolites.get_by_id(f"{met}_e"),
                     type='exchange',
                     reaction_id = f"EX_{met}_e",
                     lb = -1, ub= 1000) #set lb to -1 mmol/gcdw/h to have some excess but not endless amounts

In [16]:
#check the new compounds have been added to the defined medium
pth.medium

{'EX_cl_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_fe2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_o2_e': 0.1,
 'EX_co2_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_btn_e': 1,
 'EX_thmpp_e': 1,
 'EX_sprm_e': 1,
 'EX_spmd_e': 1,
 'EX_adocbl_e': 1,
 'EX_10fthf_e': 1,
 'EX_fe3_e': 1,
 'EX_pheme_e': 1}

Add acetate

In [17]:
medium = pth.medium
medium["EX_ac_e"] = 1000.0
pth.medium = medium

pth.medium

{'EX_cl_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_fe2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_o2_e': 0.1,
 'EX_co2_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_ac_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_btn_e': 1,
 'EX_thmpp_e': 1,
 'EX_sprm_e': 1,
 'EX_spmd_e': 1,
 'EX_adocbl_e': 1,
 'EX_10fthf_e': 1,
 'EX_fe3_e': 1,
 'EX_pheme_e': 1}

In [18]:
#check the model can grow anaerobically
pth.reactions.EX_o2_e.bounds=(0,1000)

In [19]:
pth.optimize().objective_value

0.12435311994048392

In [20]:
#confirm we only take up trace elements of the co-factors
pth.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
10fthf_e,EX_10fthf_e,4.441E-05,20,0.00%
adocbl_e,EX_adocbl_e,2.476E-05,72,0.00%
btn_e,EX_btn_e,2.221E-07,10,0.00%
ca2_e,EX_ca2_e,0.0003497,0,0.00%
fe2_e,EX_fe2_e,2.476E-05,0,0.00%
fe3_e,EX_fe3_e,0.003775,0,0.00%
glc__D_e,EX_glc__D_e,10,6,99.99%
h2o_e,EX_h2o_e,6.989,0,0.00%
k_e,EX_k_e,0.07863,0,0.00%
mg2_e,EX_mg2_e,0.01088,0,0.00%


So we have now inserted the acetone pathway, as well as KO'ed the LDH reaction. we slso supplied the different cofactors and allowed their uptake. I will then save this model so we can use it in furhter dFBA. 

Save model

In [21]:
cobra.io.write_sbml_model(pth,join(data_dir,"p-thermo_acetone_anaerobic.xml"))